In [ ]:
loaded_modules=0

In [ ]:
import os
import sys
import numpy as np
import swyft
import pickle
import matplotlib.pyplot as plt
import torch
import importlib
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor
import wandb
import copy

In [ ]:
results_dir_local=os.getcwd()

In [ ]:
config_trial = None
physics_trial = None
store_trial = None
net_trial = None


in_archive = True if len(results_dir_local.split("/archive/")) != 1 else False
if config_trial == None:
    config_path = results_dir_local+"/config_variables.pickle"
elif config_trial>-1:
    config_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(physics_trial)+"/config_variables.pickle"
else:
    config_path = results_dir_local.split("/archive/")[0]+"/config_variables.pickle"
if physics_trial == None:
    physics_path = results_dir_local+"/physics_variables.pickle"
elif physics_trial>-1:
    physics_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(physics_trial)+"/physics_variables.pickle"
else:
    physics_path = results_dir_local.split("/archive/")[0]+"/physics_variables.pickle"
if store_trial == None:
    store_path = results_dir_local+"/sim_output/store" if not in_archive else results_dir_local+"/store"
elif store_trial>-1:
    store_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(store_trial)+"/store"
else:
    store_path = results_dir_local.split("/archive/")[0]+"/sim_output/store"
if net_trial == None:
    net_path = results_dir_local+"/train_output/net" if not in_archive else results_dir_local+"/net"
elif net_trial>-1:
    net_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(net_trial)+"/net"
else:
    net_path = results_dir_local.split("/archive/")[0]+"/train_output/net"

if loaded_modules:
    importlib.reload(network)
    importlib.reload(ALP_quick_sim)
    importlib.reload(alp_swyft_simulator)
    importlib.reload(param_func)
    
sys.path.append(results_dir_local)
import ALP_quick_sim
ALP_sim = ALP_quick_sim.ALP_sim
import alp_swyft_simulator 
ALP_SWYFT_Simulator = alp_swyft_simulator.ALP_SWYFT_Simulator
sys.path.append(physics_path)
import param_function as param_func
param_function = param_func.param_function
sys.path.append(net_path)
import network 
Network = network.Network
loaded_modules=1   

In [ ]:
with open(physics_path, 'rb') as file:
    config_objects = pickle.load(file)
for key in config_objects.keys():
    locals()[key] = config_objects[key]
with open(config_path, 'rb') as file:
    config_objects = pickle.load(file)
for key in config_objects.keys():
    locals()[key] = config_objects[key]
results_dir = results_dir_local

In [ ]:
sim = ALP_SWYFT_Simulator(A, bounds)

In [ ]:
bounds

In [ ]:
colors = [
    'r',
    '#FFA500',
    'y',
    'g',
    'b',
    'k'
]


truths = [A.null_params]
print(truths)

In [ ]:
observations = []
for tru in truths:
    sim.A.configure_model(ALP_seed=0)
    obs_exp=sim.generate_exp(tru)
    obs_data = sim.generate_data(obs_exp,tru)
    observations.append(swyft.Sample(data=obs_data, exp=obs_exp, params=tru))
sim.A.configure_model(ALP_seed=None)

In [ ]:
store = swyft.ZarrStore(store_path + "/" + store_name)
samples = store.get_sample_store()

In [ ]:
len(store)

In [ ]:
plt.figure(figsize=(8,2))
for i in range(len(bounds)):
    plt.subplot(1,len(samples['params'][i]),i+1)
    plt.hist(samples['params'][:,i])

In [ ]:
len(samples['params'][0])

In [ ]:
A.configure_plot(xmin=None, xmax=None, ymin=None,ymax=None,legend=0, logx=1, logy=0)
for i in range(min(len(samples),100)):
    A.configure_model(params=samples[i]['params'])
    A.import_counts(obs=samples[i]['data'],exp=samples[i]['exp'])
    new_fig = 1 if i==0 else 0
    A.compute_case(new_fig=new_fig,new_counts=0,color=None,color_obs=None,linestyle="-",
                   label_exp=0,label_obs=False, errorbands=False, errors=False, plot_exp=0, transparency=0.7)
    
for i,obs in enumerate(observations):
    A.configure_model(params=obs['params'])
    A.import_counts(obs=obs['data'],exp=obs['exp'])
    shade = i/(len(observations)+1)
    A.compute_case(new_fig=0,new_counts=0,color='k',color_obs=colors[i],linestyle_obs="-",label_exp=0,label_obs=0,
                   errorbands=0, errors=0, plot_exp=0)
    A.compute_case(new_fig=0,new_counts=0,color=colors[i],color_obs=colors[i],linestyle=":",label_exp=0,label_obs=0,
                        errorbands=0, errors=0, plot_obs=0) 

In [ ]:
prior_samples = sim.sample(100_000, targets=['params'])

In [ ]:
new_net = Network(nbins=A.nbins, marginals=POI_indices, param_names=A.param_names)
new_net.load_state_dict(torch.load(net_path+'/trained_network.pt'))

In [ ]:
lr_monitor = LearningRateMonitor(logging_interval='step')
trainer = swyft.SwyftTrainer(
    accelerator = 'cpu', precision = 64, #logger=wandb_logger, # min_epochs =25,
    callbacks = [lr_monitor], log_every_n_steps=10,
)

In [ ]:
logratios = list(np.zeros(len(truths)))
for j in range(len(truths)):
    plt.figure(figsize=(10,2))
    logratios[j] = trainer.infer(
                            new_net,
                            observations[j],
                            prior_samples
                            )



In [ ]:
for j in range(len(truths)):
    swyft.plot_posterior(logratios[j], 
                         np.array(A.param_names)[POI_indices], 
                         truth={A.param_names[i]:truths[j][i] for i in POI_indices}, 
                         color_truth=colors[j]
                        )

In [ ]:
for j in range(len(truths)):
    swyft.plot_corner(logratios[j], 
                      np.array(A.param_names)[POI_indices], 
                      truth={A.param_names[i]:truths[j][i] for i in POI_indices},
                      color_truth=colors[j],
                      smooth=3
                     )

In [ ]:
def find_POI_matches(POI_indices, target_values, ns=None, bounds=bounds, samples=samples):
    if not ns: ns = np.ones(len(POI_indices))*5
    matches = list(np.zeros(len(POI_indices)))#,dtype=int))
    max_is = list(np.zeros(len(POI_indices)))#,dtype=int))
    for i in range(len(POI_indices)): 
        matches[i] = np.full(ns[i], None)
        max_is[i] = 0
    for samp_i in np.arange(0,len(samples)):
        for j in range(len(POI_indices)):
            current_max = abs(samples["params"][matches[j][max_is[j]]][POI_indices[j]]-target_values[j]) if not np.any(matches[j]==None) else np.inf
            if abs(samples['params'][samp_i][POI_indices[j]]-target_values[j]) <= current_max:
                within_bounds = 1
                if not ( np.any(samples["params"][samp_i,:] < np.array(bounds)[:,0]) or np.any(samples["params"][samp_i,:] > np.array(bounds)[:,1]) ):
                    nones = np.where(matches[j]==None)[0]
                    if len(nones)==0:
                        matches[j][max_is[j]] = int(samp_i)
                        max_is[j] = int(np.argmax(abs(samples["params"][matches[j]][:,POI_indices[j]]-target_values[j]))) 
                    else:
                        max_is[j] = int(nones[0])
                        matches[j][max_is[j]] = int(samp_i)
                        if max_is[j]==ns[j]-1: matches[j] = [int(x) for x in matches[j]]

    for i in range(len(POI_indices)): 
        matches[i] = np.array(matches[i])[np.array(matches[i])!=None]
        if len(matches[i])==0:
            raise ValueError("There were no matches to these bounds!")
        
    return matches

def plot_POI_matches(POI_indices, target_values, ns=None, colors=['r','g','c','b'], bounds=bounds, samples=samples):
    if not ns: ns = np.ones(len(POI_indices))*5
    matches = list(np.zeros(len(POI_indices)))
    matches = find_POI_matches(POI_indices, target_values, ns=ns, bounds=bounds, samples=samples)
    plt.figure(figsize=(8,2))
    for i in range(len(POI_indices)):
        plt.subplot(1,len(POI_indices),i+1)
        plt.hist(samples['params'][matches[i],POI_indices[i]], range=bounds[POI_indices[i]],bins=50)
        plt.title(A.param_names[POI_indices[0]] + " = " + str(target_values[i]))
    A.configure_plot(xmin = None, xmax = None, ymin=None, ymax=None, logx=1, legend = 1)
    A.compute_case(new_fig=1,null=1,plot_obs=0,color="k",linestyle="--", label_exp=1)
    for i, _ in enumerate(matches):
        for j, match in enumerate(matches[i]):
            A.configure_model(params=samples["params"][match])
            A.import_counts(exp=samples["exp"][match])
            A.compute_case(new_fig=0,new_counts=0,plot_obs=0,color=colors[i],label_exp=0, errorbands=False, transparency=0.5)    
    
    return matches

In [ ]:
%%time
n_vis = 50
bounds_vis = copy.deepcopy(bounds)
target_values = copy.deepcopy(bounds)
target_values[0][1] = np.log10(10)
target_values[1][1] = np.log10(0.8)
n_samples = min(100_000,n_sim_train)
for index in POI_indices:
    matches = plot_POI_matches([index, index],[target_values[index][0], target_values[index][1]], [n_vis,n_vis], bounds=bounds_vis, samples=samples[:n_samples])
    A.import_counts(exp=observations[0]["exp"])
    A.compute_case(new_fig=0,new_counts=0,plot_obs=0,color='k',label_exp=0, errorbands=False, transparency=0.5)

In [ ]:
# %%time
# n_vis = 20
# margin = 0.05
# bounds_vis = copy.deepcopy(bounds)
# for b, bound in enumerate(bounds_vis):
#     bounds_vis[b][0] = truths[0][b]-margin*(bounds[b][1]-bounds[b][0])
#     bounds_vis[b][1] = truths[0][b]+margin*(bounds[b][1]-bounds[b][0])
# for index in [0,1]:
#     bounds_vis[index]=copy.deepcopy(bounds[index])
# print(truths)
# print(bounds_vis)
# target_values = copy.deepcopy(bounds)
# target_values[0][1] = np.log10(10)
# target_values[1][1] = np.log10(0.8)
# n_samples = min(100_000,n_sim_train)
# for index in POI_indices:
#     matches = plot_POI_matches([index, index],[target_values[index][0], target_values[index][1]], [n_vis,n_vis], bounds=bounds_vis, samples=samples[:n_samples])
#     A.import_counts(exp=observations[0]["exp"])
#     A.compute_case(new_fig=0,new_counts=0,plot_obs=0,color='k',label_exp=0, errorbands=False, transparency=0.5)

In [ ]:
coverage_samples = trainer.test_coverage(new_net, samples[-min(n_sim_coverage,100):], prior_samples[:min(len(prior_samples),10_000)])

In [ ]:
columns = 3
columns = min(len(POI_indices),columns)
rows = int(np.ceil(len(POI_indices)/columns))
#fix, axes = plt.subplots(rows, columns, figsize = (12, 4*rows))
fix = plt.figure(figsize = (12, 4*rows))
for i in range(len(POI_indices)):
    j = int(i/columns)
    k = i%columns
    fix.add_subplot(rows, columns, i+1)
    swyft.plot_pp(coverage_samples, 
                  np.array(A.param_names)[POI_indices][i], 
                  ax = fix.axes[i])
plt.tight_layout()

In [ ]:
columns = 3
columns = min(len(POI_indices),columns)
rows = int(np.ceil(len(POI_indices)/columns))
fix = plt.figure(figsize = (12, 4*rows))
for i in range(len(POI_indices)):
    j = int(i/columns)
    k = i%columns
    fix.add_subplot(rows, columns, i+1)
    swyft.plot_zz(coverage_samples, 
                  np.array(A.param_names)[POI_indices][i], 
                  ax = fix.axes[i])
plt.tight_layout()

In [ ]:
marginals_2d = []
for i, el in enumerate(POI_indices[:-1]):
    for j in np.arange(i+1,len(POI_indices)):
        marginals_2d.append( (el,POI_indices[j]) )
marginals_2d = tuple(marginals_2d)

In [ ]:
columns = 3
columns = min(len(marginals_2d),columns)
rows = int(np.ceil(len(marginals_2d)/columns))
fix = plt.figure(figsize = (12, 4*rows))
for i in range(len(marginals_2d)):
    j = int(i/columns)
    k = i%columns
    fix.add_subplot(rows, columns, i+1)
    swyft.plot_pp(coverage_samples, 
                  [[np.array(param_names)[POI_indices][j] for j in marginal] for marginal in marginals_2d ][i], 
                  ax = fix.axes[i])
plt.tight_layout()

In [ ]:
columns = 3
columns = min(len(marginals_2d),columns)
rows = int(np.ceil(len(marginals_2d)/columns))
fix = plt.figure(figsize = (12, 4*rows))
for i in range(len(marginals_2d)):
    j = int(i/columns)
    k = i%columns
    fix.add_subplot(rows, columns, i+1)
    swyft.plot_zz(coverage_samples, 
                  [[np.array(param_names)[POI_indices][j] for j in marginal] for marginal in marginals_2d ][i], 
                  ax = fix.axes[i])
plt.tight_layout()